In [ ]:
import numpy as np
import pandas as pd

# DATA

In [ ]:
file_path = '../../src/data/temp/processed_zbp_detail_data.csv'
zbp_detail = pd.read_csv(file_path)
zbp_detail.head()

In [ ]:
file_path = '../../src/data/temp/processed_zbp_totals_data.csv'
zbp_totals = pd.read_csv(file_path)
zbp_totals.head()

# Features

* presence of anchor business(es) (eg, measured by the number of large businesses or combination of large business and industry)
* % tech
* % bio
* % restaurant
* % small business
* population
* % population working (labor force %)


# Industry Percentages

In [ ]:
zip_est = zbp_detail.groupby(['zip', 'year'])['est'].sum()
zip_naics = zbp_detail.groupby(['zip', 'year', 'naics'])['est'].sum()
naics_percentages = (zip_naics/zip_est).unstack().fillna(value=0).add_prefix('naics_').add_suffix('_pct').reset_index()
naics_percentages

# Establishment Size Percentages

In [ ]:
est_size_percentages = zbp_detail.groupby(['zip', 'year'])[['n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000']].sum()
est_size_percentages = est_size_percentages.apply(lambda ser: ser/zip_est, axis=0).add_suffix('_pct').reset_index()
est_size_percentages

# Compile Features

In [ ]:
zbp_totals_with_features = zbp_totals.merge(naics_percentages, on=['zip', 'year'])
zbp_totals_with_features = zbp_totals_with_features.merge(est_size_percentages, on=['zip', 'year'])
zbp_totals_with_features.to_csv('../../src/data/temp/zbp_totals_with_features.csv', index=False)
zbp_totals_with_features # 1523 rows instead of 1705 because there are more (zip, year) pairs in totals than in details